#  Script: Reproject dataset with rasterio

#### Florian Beyer

2019-03-13

Version 0.1

In [1]:
# required packages
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

import glob
import os
import sys

import numpy as np

In [2]:
# where are the images?
infolder = r'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei'

# where to save the reprojected images
outfolder = r'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\005_Convert_UTM32_to_UTM33\\'

# target coordinate system as epsg code (http://spatialreference.org/)
target_crs = 'EPSG:32633'

In [3]:
def reproject_tiff(image, dst_crs, save_path, out_filename):
    '''
    https://github.com/mapbox/rasterio/blob/master/docs/topics/reproject.rst
    
    Reprojecting a GeoTIFF dataset
    Reprojecting a GeoTIFF dataset from one coordinate reference system is a common use case.
    Rasterio provides a few utilities to make this even easier:
        - transform_bounds() transforms the bounding coordinates of the source raster to the target
          coordinate reference system, densifiying points along the edges to account for non-linear
          transformations of the edges.

        - calculate_default_transform() transforms bounds to target coordinate system, calculates
          resolution if not provided, and returns destination transform and dimensions.
    
    image: rasterio dataset (multiband image)
    dst_crs: target cordinatesystem as epsg code
    save_path: string with the path where the image will be saved
    out_filename: filename of the reprojected image
    '''
    
    src = image
    transform,width,height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
    # get infromation of the initial dataset
    
    kwargs = src.meta.copy() #copy metadata

    kwargs.update({ # update metadata with the target coordinate system
        'crs': dst_crs,
        'transform':transform,
        'width': width,
        'height': height
    })
    
    with rasterio.open(save_path+out_filename, 'w', **kwargs) as dst:
        '''
        Writes the nee image with the target coordinate system
        '''
        for i in range(1, src.count + 1):
            rasterio.warp.reproject(
            source=rasterio.band(src, i),
            destination=rasterio.band(dst,i),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=rasterio.warp.Resampling.nearest # choose a method
            )

In [4]:
# search all datasets in UTM32
temp = os.path.join(infolder, '*T32*.tif') ### Attention! hard coded --> *T32
files = glob.glob(temp)
temp = None
files

['N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180420_S2B_MSIL2A_T32UPE_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180420_S2B_MSIL2A_T32UPF_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180505_S2A_MSIL2A_T32UPF_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180507_S2B_MSIL2A_T32UPE_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180507_S2B_MSIL2A_T32UPF_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180520_S2B_MSIL2A_T32UPE_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180520_S2B_MSIL2A_T32UPF_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180530_S2B_MSIL2A_T32UPE_s.tif',
 'N:\\\\Projekt_PROSPER_RO\\\\Data_RS\\\\Sentinel2\\\\004_Subsets_wolkenfrei\\20180530_S2B_MSIL2

In [5]:
# read all dataset with rasterio
files_to_reproject = []
for i in files:
    src = rasterio.open(i)
    files_to_reproject.append(src)
files_to_reproject

[<open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei\20180420_S2B_MSIL2A_T32UPE_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei\20180420_S2B_MSIL2A_T32UPF_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei\20180505_S2A_MSIL2A_T32UPF_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei\20180507_S2B_MSIL2A_T32UPE_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei\20180507_S2B_MSIL2A_T32UPF_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei\20180520_S2B_MSIL2A_T32UPE_s.tif' mode='r'>,
 <open DatasetReader name='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\004_Subsets_wolkenfrei\20180520_S2B_MSIL2A_T32UPF_s.tif' mode='r'>,
 <open DatasetReader name='

In [6]:
for i in range(len(files)):
    '''
    loop over all rasterio datsets in order to reproject all images
    '''
    filename = files[i][-32:-4]+'_reproject.tif' # attention! it's hard coded
    reproject_tiff(image=files_to_reproject[i],
                   dst_crs=target_crs,
                   save_path=outfolder,
                   out_filename=filename)
    print filename + ' reprojected to ' + target_crs

20180420_S2B_MSIL2A_T32UPE_s_reproject.tif reprojected to EPSG:32633
20180420_S2B_MSIL2A_T32UPF_s_reproject.tif reprojected to EPSG:32633
20180505_S2A_MSIL2A_T32UPF_s_reproject.tif reprojected to EPSG:32633
20180507_S2B_MSIL2A_T32UPE_s_reproject.tif reprojected to EPSG:32633
20180507_S2B_MSIL2A_T32UPF_s_reproject.tif reprojected to EPSG:32633
20180520_S2B_MSIL2A_T32UPE_s_reproject.tif reprojected to EPSG:32633
20180520_S2B_MSIL2A_T32UPF_s_reproject.tif reprojected to EPSG:32633
20180530_S2B_MSIL2A_T32UPE_s_reproject.tif reprojected to EPSG:32633
20180530_S2B_MSIL2A_T32UPF_s_reproject.tif reprojected to EPSG:32633
20180606_S2B_MSIL2A_T32UPE_s_reproject.tif reprojected to EPSG:32633
20180606_S2B_MSIL2A_T32UPF_s_reproject.tif reprojected to EPSG:32633
20180704_S2A_MSIL2A_T32UPE_s_reproject.tif reprojected to EPSG:32633
20180704_S2A_MSIL2A_T32UPF_s_reproject.tif reprojected to EPSG:32633
20180706_S2B_MSIL2A_T32UPE_s_reproject.tif reprojected to EPSG:32633
20180706_S2B_MSIL2A_T32UPF_s_repro

In [7]:
# for one single image:
src = rasterio.open(r'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\003_Subsetting\\20180505_S2A_MSIL2A_T32UPE_s.tif')

In [8]:
# for one single image:
reproject_tiff(image=src,
               dst_crs=target_crs,
               save_path='N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\005_Convert_UTM32_to_UTM33\\',
               out_filename='20180505_S2A_MSIL2A_T32UPE_s_reproject.tif')